# Demo of `create_climpred_data`

This demo demonstrates how you setup your raw output from a climate model to match `climpred`'s expectations.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import climpred

In [2]:
from climpred.create_climpred_data import load_hindcast, climpred_preprocess_internal, get_path

Assuming your raw model output is stored in multiple files per member and initialization, `load_hindcast` is a nice wrapper function based on `get_path` designed for the output format of `MPI-ESM` to aggregated all hindcast output into one file as expected by `climpred`.

The basic idea is to look over the output of all members and concatinate, then loop over all initializations and concatinate. Before concatination, it is important to align the `time` dimension.

To reduce the data size, use the `preprocess` function wisely in combination with `climpred_preprocess_internal`, e.g. additionally extracting only a certain region or only few variables for a multi-variable input file as in MPI-ESM standard output.

In [3]:
get_path??

Signature:
get_path(
    dir_base_experiment='/work/bm1124/m300086/CMIP6/experiments',
    member=1,
    init=1960,
    model='hamocc',
    output_stream='monitoring_ym',
    timestr='*1231',
    ending='nc',
    verbose=False,
)
Source:   
def get_path(
    dir_base_experiment="/work/bm1124/m300086/CMIP6/experiments",
    member=1,
    init=1960,
    model="hamocc",
    output_stream="monitoring_ym",
    timestr="*1231",
    ending="nc",
    verbose=False,
):
    """Get the path of a file of for MPI-ESM standard output file names and directory."""
    # get experiment_id
    dirs = _os.listdir(dir_base_experiment)
    experiment_id = [
        x for x in dirs if (f"{init}" in x and "r" + str(member) + "i" in x)
    ]
    assert len(experiment_id) == 1
    experiment_id = experiment_id[0]
    dir_outdata = f"{dir_base_experiment}/{experiment_id}/outdata/{model}"
    path = f"{dir_outdata}/{experiment_id}_{model}_{output_stream}_{timestr}.{ending}"
    if verbose:
        print(f"Path: 

In [4]:
climpred_preprocess_internal??

Signature: climpred_preprocess_internal(ds)
Source:   
def climpred_preprocess_internal(ds):
    """CMIP6 DCPP preprocessing before the aggreatations of intake-esm happen."""
    # set time to integers starting at 1
    ds['time'] = np.arange(1, 1 + ds.time.size)
    return ds
File:      /work/mh0727/m300524/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/climpred-1.2.0.post5+dirty-py3.6.egg/climpred/create_climpred_data.py
Type:      function


In [5]:
load_hindcast??

Signature:
load_hindcast(
    inits=range(1961, 1965),
    members=range(1, 3),
    preprocess=None,
    lead_offset=1,
    parallel=True,
    load_grb_func=None,
    **get_path_kwargs,
)
Source:   
def load_hindcast(
    inits=range(1961, 1965),
    members=range(1, 3),
    preprocess=None,
    lead_offset=1,
    parallel=True,
    load_grb_func=None,
    **get_path_kwargs,
):
    """Load multi-member, multi-initialization hindcast experiment into one xr.Dataset compatible with `climpred`."""
    init_list = []
    for init in inits:
        print(f"Processing init {init} ...")
        member_list = []
        for member in members:
            # get path p
            p = get_path(member=member, init=init, **get_path_kwargs)
            ending = get_path_kwargs.get('ending', 'nc')
            # open all leads for specified member and init
            if ending is 'nc':
                member_ds = xr.open_mfdataset(
                    p,
                    combine="nested",
        

In [6]:
def preprocess_1var(ds, v="global_primary_production"):
    return ds[v].to_dataset(name=v).squeeze()

In [7]:
ds = load_hindcast(preprocess=preprocess_1var).load()
ds.coords

Processing init 1961 ...
Processing init 1962 ...
Processing init 1963 ...
Processing init 1964 ...


Coordinates:
    lon      float64 0.0
    depth    float64 0.0
    lat      float64 0.0
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10
  * member   (member) int64 1 2
  * init     (init) int64 1961 1962 1963 1964

In [8]:
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

<xarray.Dataset>
Dimensions:                    (lead: 10)
Coordinates:
    lon                        float64 0.0
    depth                      float64 0.0
    lat                        float64 0.0
  * lead                       (lead) int64 1 2 3 4 5 6 7 8 9 10
Data variables:
    global_primary_production  (lead) float64 -0.1276 0.593 ... 0.09196 0.5038
Attributes:
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     4
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    units:                         None
    created:                       2020-01-06 15:00:30

# `intake-esm` for cmorized output

In case you have access to cmorized output of CMIP experiments, consider using `intake-esm`. With the `preprocess` function you can align the `time` dimension of the output. Finally, `climpred_preprocess_post` only renames.

In [9]:
from climpred.create_climpred_data import climpred_preprocess_post, climpred_preprocess_internal

In [10]:
import intake

In [11]:
col_url = "/home/mpim/m300524/intake-esm-datastore/catalogs/mistral-cmip6.json"
col = intake.open_esm_datastore(col_url)

In [12]:
# load 2 members for 2 inits from one model
query = dict(experiment_id=[
    'dcppA-hindcast'], table_id='Amon', member_id=['r1i1p1f1', 'r2i1p1f1'], dcpp_init_year=[1970, 1971], variable_id='tas', source_id='MPI-ESM1-2-HR')
cat = col.search(**query)
cdf_kwargs = {'chunks': {'time': 12}, 'decode_times': False}

In [13]:
def preprocess(ds):
    # extract tiny spatial and temporal subset
    ds = ds.isel(lon=[50, 51, 52], lat=[50, 51, 52],
                 time=np.arange(12 * 2))
    # make time dim identical
    ds = climpred_preprocess_internal(ds)
    return ds

In [17]:
dset_dict = cat.to_dataset_dict(
    cdf_kwargs=cdf_kwargs, preprocess=preprocess)
# get first dict value
ds = dset_dict[list(dset_dict.keys())[0]].load()
ds.coords

Progress: |███████████████████████████████████████████████████████████████████████████████| 100.0% 

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
             
--> There are 1 group(s)


Coordinates:
    height          float64 2.0
  * lon             (lon) float64 46.88 47.81 48.75
  * dcpp_init_year  (dcpp_init_year) float64 1.97e+03 1.971e+03
  * time            (time) int64 1 2 3 4 5 6 7 8 9 ... 17 18 19 20 21 22 23 24
  * lat             (lat) float64 -42.55 -41.61 -40.68
  * member_id       (member_id) <U8 'r1i1p1f1' 'r2i1p1f1'

In [18]:
ds = climpred_preprocess_post(ds)
ds.coords

Coordinates:
    height   float64 2.0
  * lon      (lon) float64 46.88 47.81 48.75
  * init     (init) float64 1.97e+03 1.971e+03
  * lead     (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 15 16 17 18 19 20 21 22 23 24
  * lat      (lat) float64 -42.55 -41.61 -40.68
  * member   (member) <U8 'r1i1p1f1' 'r2i1p1f1'

In [19]:
climpred.prediction.compute_perfect_model(ds, ds.rename({'lead':'time'}))

/work/mh0727/m300524/miniconda3/envs/climpred-dev/lib/python3.6/site-packages/xskillscore/core/np_deterministic.py:182: RuntimeWarning: invalid value encountered in true_divide
  r = r_num / r_den


<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 3, lead: 24, lon: 3)
Coordinates:
    height     float64 2.0
  * lon        (lon) float64 46.88 47.81 48.75
  * lead       (lead) int64 1 2 3 4 5 6 7 8 9 10 ... 16 17 18 19 20 21 22 23 24
  * lat        (lat) float64 -42.55 -41.61 -40.68
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (lead, bnds) float64 nan nan nan nan nan ... nan nan nan nan nan
    lon_bnds   (lead, lon, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    lat_bnds   (lead, lat, bnds) float64 nan nan nan nan nan ... nan nan nan nan
    tas        (lead, lat, lon) float32 0.933643 0.88438606 ... -0.8581106
Attributes:
    grid:                          spectral T127; 384 x 192 longitude/latitude
    external_variables:            areacella
    forcing_index:                 1
    Conventions:                   CF-1.7 CMIP-6.2
    cmor_version:                  3.5.0
    experiment:                    hindcast initialized based on observations...
    contact:                       cmip6-mpi-esm@dkrz.de
    initialization_index:          1
    physics_index:                 1
    frequency:                     mon
    institution:                   Max Planck Institute for Meteorology, Hamb...
    variable_id:                   tas
    activity_id:                   DCPP
    mip_era:                       CMIP6
    source_type:                   AOGCM
    project_id:                    CMIP6
    table_info:                    Creation Date:(09 May 2019) MD5:e6ef8ececc...
    tracking_id:                   hdl:21.14100/f41d2fe5-bb1c-49d0-8afa-0cb9e...
    title:                         MPI-ESM1-2-HR output prepared for CMIP6
    experiment_id:                 dcppA-hindcast
    branch_method:                 standard lagged initialization
    grid_label:                    gn
    source:                        MPI-ESM1.2-HR (2017): \naerosol: none, pre...
    product:                       model-output
    source_id:                     MPI-ESM1-2-HR
    history:                       2019-09-06T14:20:04Z ; CMOR rewrote data t...
    references:                    MPI-ESM: Mauritsen, T. et al. (2019), Deve...
    license:                       CMIP6 model data produced by MPI-M is lice...
    data_specs_version:            01.00.30
    table_id:                      Amon
    nominal_resolution:            100 km
    institution_id:                MPI-M
    realm:                         atmos
    intake_esm_varname:            tas
    prediction_skill:              calculated by climpred https://climpred.re...
    skill_calculated_by_function:  compute_perfect_model
    number_of_initializations:     2
    number_of_members:             2
    metric:                        pearson_r
    comparison:                    m2e
    units:                         None
    created:                       2020-01-06 15:01:34